# Parallel RAG Notebook
This notebook uses **GPT-5** and **FAISS** to perform parallel retrieval and generation on Databricks.

In [0]:
%pip install openai faiss-cpu pandas
dbutils.library.restartPython()

In [0]:
import os
import pandas as pd
import numpy as np
import faiss
from openai import OpenAI
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType

# Retrieve API Key from Secrets
os.environ['OPENAI_API_KEY'] = dbutils.secrets.get("solution-accelerator-cicd", "openai_api")

client = OpenAI()
EMBEDDING_MODEL = "text-embedding-3-small"
LLM_MODEL = "gpt-5"

In [0]:
# Create Local Documents
doc_dir = "/documents"
if not os.path.exists(doc_dir):
    os.makedirs(doc_dir)

agents = {
    "jett.txt": "Jett is a duelist. Abilities: Tailwind, Cloudburst.",
    "sage.txt": "Sage is a sentinel. Abilities: Healing Orb, Resurrection.",
    "sova.txt": "Sova is an initiator. Abilities: Recon Bolt, Owl Drone.",
    "phoenix.txt": "Phoenix is a duelist. Abilities: Curveball, Blaze.",
    "viper.txt": "Viper is a controller. Abilities: Toxic Screen, Poison Cloud."
}

for filename, content in agents.items():
    with open(os.path.join(doc_dir, filename), "w") as f:
        f.write(content)

In [0]:
# Indexing with FAISS
documents = [open(os.path.join(doc_dir, f)).read() for f in os.listdir(doc_dir)]

def get_emb(texts):
    res = client.embeddings.create(input=texts, model=EMBEDDING_MODEL)
    return np.array([d.embedding for d in res.data]).astype('float32')

embeddings = get_emb(documents)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [0]:
# NEW: Broadcast variables to all cluster nodes
broadcast_docs = sc.broadcast(documents)
broadcast_index = sc.broadcast(index)

# Ensure workers have access to the API key in their environment
os.environ['OPENAI_API_KEY'] = dbutils.secrets.get("solution-accelerator-cicd", "openai_api")

In [0]:
# Updated Parallel RAG via Spark
queries = ["Jett abilities?", "Sage role?", "Sova tools?", "Phoenix flash?", "Viper gas?"]
df_q = spark.createDataFrame(pd.DataFrame(queries, columns=["question"]))

def ask_gpt_worker(q):
    # Initialize a NEW client inside the worker to avoid pickling errors
    worker_client = OpenAI()
    
    # Access the broadcasted variables
    local_index = broadcast_index.value
    local_docs = broadcast_docs.value
    
    # 1. Retrieval (using broadcasted index)
    res_emb = worker_client.embeddings.create(input=[q], model=EMBEDDING_MODEL)
    vec = np.array([res_emb.data[0].embedding]).astype('float32')
    
    _, idx = local_index.search(vec, 1)
    ctx = local_docs[idx[0][0]]
    
    # 2. Generation (using GPT-5)
    ans = worker_client.chat.completions.create(
        model=LLM_MODEL,
        messages=[
            {"role": "system", "content": "Use context to answer concisely."},
            {"role": "user", "content": f"Context: {ctx}\nQuestion: {q}"}
        ]
    )
    return ans.choices[0].message.content

@pandas_udf(StringType())
def parallel_rag(s: pd.Series) -> pd.Series:
    # This runs on cluster executors in parallel
    return s.apply(ask_gpt_worker)

# Display results
display(df_q.withColumn("gpt_5_answer", parallel_rag(df_q["question"])))